In [5]:
import random
import re
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import re
import numpy as np

In [6]:
credentials = {"client_id": "",
    "client_secret": ""}

client_id = credentials['client_id']
client_secret = credentials['client_secret']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [7]:
def getAudioAnalysis(uri): #returns a dict
    dictA = {}
    dictA['song_id'] = uri
    words = ['analysis_sample_rate']

    analysis = sp.audio_analysis(uri)
    for instan in analysis["track"]:
        if instan in words:
            val = analysis["track"][instan]
            dictA[instan] = val
    return dictA

In [8]:
def getAudioFeatures(uri): #returns a dict
    dictA = {}
    words = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'key', 'loudness', 'mode', 'time_signature', 'tempo', 'time_signature', 'valence', 'duration_ms']

    analysis = sp.audio_features(uri)
    for instan in analysis[0]:
        if instan in words:
            if instan == 'duration_ms':
                val = (analysis[0][instan]/1000)
                dictA['duration'] = val
                continue
            if instan == 'tempo':
                val = analysis[0][instan]
                dictA['bpm'] = val
                continue
            else: 
                val = analysis[0][instan]
                dictA[instan] = val
    return dictA

In [9]:
def levenshtein_distance(search, result):
  
    result = result.lower()

    m = len(search)
    n = len(result)
    d = [[0 for j in range(n + 1)] for i in range(m + 1)]

    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j

    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if search[i - 1] == result[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                
                d[i][j] = min(d[i - 1][j] + 1, d[i][j - 1] + 1, d[i - 1][j - 1] + 1) #nonweighted

    return d[m][n]

In [10]:
def getTracks(searchquery):
    getTrack = sp.search(q=searchquery)
    tracks = {}
    IDlist = []
    arter = []
    trackIDlist = {}
    rez = getTrack['tracks']['items']
    arty = ""
    for i in range(len(rez)):
        try:
            album_type = rez[i]['album']['album_type']
        except:
            album_type = 'None'
        trackName = rez[i]['name']
        trackID = rez[i]['id'] #gets spotify track ID to use
        if (album_type != 'compilation'):
            if('Acoustic' not in trackName):
                regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
                if(regex.search(trackName) == None):
                    resid = [ele for ele in ['Remix','Mix','Edit'] if(ele not in trackName)]
                    if ('-' in trackName and len(resid) == 3):
                        x = trackName.split('-')
                        trackName = x[0].rstrip()
                    if(trackName not in tracks.values()):
                        tracks[trackID] = trackName #title
                        IDlist.append(trackID)
                        arter.append(rez[i]['artists'][0]['name'])
                        arty = arter[-1]
                        trackIDlist[i] = {trackID:trackName}
                    elif(trackID not in tracks.keys() and trackName in tracks.values() and arty not in arter): #need to account for artist instead of trackID
                        IDlist.append(trackID)
                        arter.append(rez[i]['artists'][0]['name'])
                        trackIDlist[i] = {trackID:trackName}
                elif('feat.' in trackName):
                    trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)
                    if ('-' in trackName):
                        x = trackName.split('-')
                        trackName = x[0].rstrip() + ' -' + x[1]
                    else:
                        trackName = trackName.rstrip()
                    resid = [ele for ele in ['Remix','Mix','Edit'] if(ele not in trackName)]
                    if (' - ' in trackName and 'Live' in trackName):
                        if(len(resid) == 3):
                            x = trackName.split(' - ')
                            trackName = x[0].rstrip()
                        else:
                            trackName += ' Remix'
                    else:
                        if(trackName not in tracks.values()):
                            tracks[trackID] = trackName #title
                            IDlist.append(trackID)
                            trackIDlist[i] = {trackID:trackName}
                elif('with' in trackName):
                    trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)
                    if ('-' in trackName):
                        x = trackName.split('-')
                        trackName = x[0].rstrip() + ' -' + x[1]
                    else:
                        trackName = trackName.rstrip()
                    resid = [ele for ele in ['Remix','Mix','Edit'] if(ele not in trackName)]
                    if (' - ' in trackName and 'Live' in trackName):
                        if(len(resid) == 3):
                            x = trackName.split(' - ')
                            trackName = x[0].rstrip()
                        else:
                            trackName += ' Remix'
                    else:
                        if(trackName not in tracks.values()):
                            tracks[trackID] = trackName #title
                            IDlist.append(trackID)
                            trackIDlist[i] = {trackID:trackName}
                elif('and' in trackName):
                    try:
                        if ('(and %s' in trackName, [rez[i]['artists'][1]['name']]):
                            trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)
                            trackName = trackName.rstrip()
                            tracks[trackID] = trackName #title
                            IDlist.append(trackID)
                            trackIDlist[i] = {trackID:trackName}
                    except:
                        trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)
                        trackName = trackName.rstrip()
                        tracks[trackID] = trackName #title
                        IDlist.append(trackID)
                        trackIDlist[i] = {trackID:trackName}
                else:
                    if(trackName not in tracks.values()):
                        tracks[trackID] = trackName #title
                        IDlist.append(trackID)
                        trackIDlist[i] = {trackID:trackName}    
    x = trackIDlist
    y = list(x.keys())
    z = -1
    newSongList = []
    for count in y:
        asongName = list(x[count].values())[0]
        z+=1
        try:
            artistName = arter[z]
        except:
            artistName = ""
        strX = asongName + " " + artistName
        newSongList.append(strX)
    counter = 0
    for word in newSongList:
        strX = word
        distance = levenshtein_distance(searchquery, strX)
        if (distance >= len(strX.split())-1):
            return IDlist[counter]
        else:
            counter+=1

In [11]:
def duplicateSongs(listing):
    for item in listing:
        if listing.count(item) > 1:
            print("Duplicates found")
            print(item)
    print("No duplicates found")

In [12]:
#50 songs - good
songSearchBlues = [
    "Cross Road Blues Robert Johnson",                
    "I Can't Quit You Baby Willie Dixon", 
    "Red House Jimi Hendrix", 
    "Sweet Home Chicago Robert Johnson", 
    "I'm Your Hoochie Coochie Man Muddy Waters", 
    "Texas Flood Stevie Ray Vaughan", 
    "Dust My Broom Elmore James", 
    "Got My Mojo Working Muddy Waters",
    "Mannish Boy Muddy Waters",
    "Hoochie Coochie Man Muddy Waters",
    "Back Door Man Howlin' Wolf",
    "I'm a King Bee Slim Harpo",
    "Reconsider Baby Lowell Fulson",
    "The Sky Is Crying Elmore James",
    "Mystery Train Junior Parker",
    "Third Degree Eddie Boyd",
    "Broke Down Engine Blind Willie McTell",
    "Statesboro Blues Blind Willie McTell",
    "Hellhound on My Trail Robert Johnson",
    "Boom Boom John Lee Hooker",
    "Rollin' Stone Muddy Waters",
    "Wang Dang Doodle Koko Taylor",
    "I'd Rather Go Blind Etta James",
    "Big Boss Man Jimmy Reed",
    "Little Red Rooster Howlin' Wolf",
    "Black Snake Moan Blind Lemon Jefferson",
    "The Same Thing Willie Dixon",
    "St. James Infirmary Blues Cab Calloway",
    "Blues Before Sunrise Leroy Carr",
    "See That My Grave Is Kept Clean Blind Lemon Jefferson",
    "Death Letter Blues Son House",
    "Catfish Blues Robert Petway",
    "Stormy Monday T-Bone Walker",
    "Thrill Is Gone B.B. King",
    "Baby Please Don't Go Big Joe Williams",
    "Killing Floor Howlin' Wolf",
    "The Thrill Is Gone Roy Hawkins",
    "Little Wing Jimi Hendrix",
    "Good Morning Little Schoolgirl Sonny Boy Williamson",
    "Every Day I Have the Blues Pinetop Sparks",
    "Boom Boom Out Go the Lights Little Walter",
    "Born Under a Bad Sign Albert King",
    "Call It Stormy Monday Albert Collins",
    "Cherry Red Big Joe Turner",
    "Don't Throw Your Love on Me So Strong Albert King",
    "Down Home Blues Z.Z. Hill",
    "Honey Hush Albert Collins",
    "I'm Tore Down Freddie King",
    "It Hurts Me Too Elmore James",
    "Juke Little Walter",
    ]

print("Length of Blues:", len(songSearchBlues))

listerBlues = []
for songSearch in songSearchBlues:
    x = getTracks(songSearch)
    listerBlues.append(x)

duplicateSongs(listerBlues)


Length of Blues: 50
No duplicates found


In [13]:
#50 songs - good
songSearchClassical = [
    "Adagio for Strings Samuel Barber",
    "Canon in D Major Johann Pachelbel",
    "The Four Seasons - Spring Antonio Vivaldi",
    "Ave Maria Franz Schubert",
    "Requiem in D minor Wolfgang Amadeus Mozart",
    "Toccata and Fugue in D minor Johann Sebastian Bach",
    "The Nutcracker Suite Pyotr Ilyich Tchaikovsky",
    "Eine Kleine Nachtmusik Wolfgang Amadeus Mozart",
    "Adagio in G minor Tomaso Albinoni",
    "The Barber of Seville Gioachino Rossini",
    "Swan Lake Suite Pyotr Ilyich Tchaikovsky",
    "Miserere Gregorio Allegri",
    "Ride of the Valkyries Richard Wagner",
    "Messiah George Frideric Handel",
    "Turandot - Nessun dorma Giacomo Puccini",
    "The Magic Flute Wolfgang Amadeus Mozart",
    "Carmen Suite No. 1 Georges Bizet",
    "Orpheus in the Underworld Jacques Offenbach",
    "La Traviata - Libiamo ne' lieti calici Giuseppe Verdi",
    "The Blue Danube Johann Strauss II",
    "Requiem Giuseppe Verdi",
    "Cello Suite No. 1 - Prelude Johann Sebastian Bach",
    "Jesu, Joy of Man's Desiring Johann Sebastian Bach",
    "Nocturne in E-Flat Major Frédéric Chopin",
    "Eine Alpensinfonie Richard Strauss",
    "Waltz of the Flowers Pyotr Ilyich Tchaikovsky",
    "Concerto for Violin and Orchestra in D Major Ludwig van Beethoven",
    "Piano Concerto No. 21 in C Major Wolfgang Amadeus Mozart",
    "Clarinet Concerto in A Major Wolfgang Amadeus Mozart",
    "Symphony No. 9 - From the New World Antonín Dvořák",
    "Meditation from Thaïs Jules Massenet",
    "Peer Gynt Suite No. 1 Edvard Grieg",
    "The Planets - Jupiter Gustav Holst",
    "Symphony No. 40 Wolfgang Amadeus Mozart",
    "Symphony No. 5 Ludwig van Beethoven",
    "Pictures at an Exhibition Modest Mussorgsky",
    "Symphony No. 9 Ludwig van Beethoven",
    "Brandenburg Concerto No. 3 Johann Sebastian Bach",
    "O Fortuna Carl Orff",
    "Adagietto from Symphony No. 5 Gustav Mahler",
    "Rhapsody in Blue George Gershwin",
    "Hungarian Rhapsody No. 2 Franz Liszt",
    "Danse Macabre Camille Saint-Saëns",
    "Die Walküre - The Ride of the Valkyries Richard Wagner",
    "Für Elise Beethoven",
    "Boléro Ravel",
    "Symphony No. 7 - Allegretto Ludwig van Beethoven",
    "Prelude to the Afternoon of a Faun Claude Debussy",
    "Symphony No. 5 - Adagietto Gustav Mahler",
    "Rhapsody on a Theme of Paganini Sergei Rachmaninoff"
    ]

print("Length of Classical:", len(songSearchClassical))

listerClassical = []
for songSearch in songSearchClassical:
    x = getTracks(songSearch)
    listerClassical.append(x)

duplicateSongs(listerClassical)


Length of Classical: 50
No duplicates found


In [14]:
#50 songs - good
songSearchCountry = [
    "Forever After All Luke Combs",
    "The Good Ones Gabby Barrett",
    "Settling Down Miranda Lambert",
    "One Margarita Luke Bryan",
    "Minimum Wage Blake Shelton",
    "Who I Am Jessica Andrews",
    "Starting Over Chris Stapleton",
    "Long Live Florida Georgia Line",
    "Somebody Like That Tenille Arts",
    "More Than My Hometown Morgan Wallen",
    "Wine, Beer, Whiskey Little Big Town",
    "Made For You Jake Owen",
    "Chasing After You Ryan Hurd",
    "Lady Brett Young",
    "The Bones Maren Morris",
    "Drunk (And I Don't Wanna Go Home) Elle King",
    "Breaking Up Was Easy In The 90's Sam Hunt",
    "Beers And Sunshine Darius Rucker",
    "Better Together Luke Combs",
    "I Hope You're Happy Now Carly Pearce",
    "All My Favorite People Maren Morris",
    "Tequila Little Time Jon Pardi",
    "Whiskey Glasses Morgan Wallen",
    "Heartless Diplo",
    "One Night Standards Ashley McBryde",
    "Hard To Forget Sam Hunt",
    "To A T Ryan Hurd",
    "You Time Scotty McCreery",
    "She Drives Me Crazy Brantley Gilbert",
    "Almost Maybes Jordan Davis",
    "Before He Cheats Carrie Underwood",
    "Chicken Fried Zac Brown Band",
    "Need You Now Lady A",
    "Alright Darius Rucker",
    "The Baby Blake Shelton",
    "Summertime Kenny Chesney",
    "Live Like You Were Dying Tim McGraw",
    "Days Go By Keith Urban",
    "She Won't be Lonely Long Clay Walker",
    "People Are Crazy Billy Currington",
    "In Color Jamey Johnson",
    "Big Deal LeAnn Rimes",
    "Mud On the Tires Brad Paisley",
    "The Good Stuff Kenny Chesney",
    "Austin Blake Shelton",
    "I Wanna Do It All Terri Clark",
    "I Wonder Kellie Pickler",
    "Hurt Johnny Cash",
    "Love's the Only House Martina McBride",
    "Back at One Mark Wills"
]

print("Length of Country:", len(songSearchCountry))

listerCountry = []
for songSearch in songSearchCountry:
    x = getTracks(songSearch)
    listerCountry.append(x)

duplicateSongs(listerCountry)

Length of Country: 50
No duplicates found


In [15]:
#50 songs - good
songSearchDisco = [
    "Stayin' Alive Bee Gees",
    "September Earth, Wind & Fire",
    "I Will Survive Gloria Gaynor",
    "Got to Be Real Cheryl Lynn",
    "Don't Stop 'Til You Get Enough Michael Jackson",
    "Boogie Wonderland Earth, Wind & Fire",
    "Night Fever Bee Gees",
    "Bad Girls Donna Summer",
    "More Than a Woman Bee Gees",
    "Super Freak Rick James",
    "Y.M.C.A. Village People",
    "You Make Me Feel (Mighty Real) Sylvester",
    "Funky Town Lipps Inc.",
    "Last Dance Donna Summer",
    "Lady Marmalade Labelle",
    "Upside Down Diana Ross",
    "Disco Inferno The Trammps",
    "Ring My Bell Anita Ward",
    "Shame Evelyn Champagne King",
    "Brick House Commodores",
    "Take Your Time (Do It Right) The S.O.S. Band",
    "He's the Greatest Dancer Sister Sledge",
    "We Are Family Sister Sledge",
    "Celebration Kool & The Gang",
    "I'm Your Boogie Man KC and the Sunshine Band",
    "Let's Groove Earth, Wind & Fire",
    "Don't Leave Me This Way Thelma Houston",
    "Ain't No Stoppin' Us Now McFadden & Whitehead",
    "The Hustle Van McCoy",
    "Boogie Nights Heatwave",
    "Rock with You Michael Jackson",
    "Le Freak Chic",
    "Dancing Queen ABBA",
    "Heart of Glass Blondie",
    "Stayin' Alive N-Trance",
    "Get Down Tonight KC and the Sunshine Band",
    "Dance, Dance, Dance (Yowsah, Yowsah, Yowsah) Chic",
    "Shake Your Groove Thing Peaches & Herb",
    "Play That Funky Music Wild Cherry",
    "Everybody Dance Chic",
    "Boogie Oogie Oogie A Taste of Honey",
    "Dance with Me Peter Brown",
    "Shake Your Body (Down to the Ground) The Jacksons",
    "You Should Be Dancing Bee Gees",
    "I Feel Love Donna Summer",
    "Dancing Machine The Jackson 5",
    "That's the Way (I Like It) KC and the Sunshine Band",
    "Funky Stuff Kool & The Gang",
    "Disco Lady Johnnie Taylor",
    "Lost in Music Sister Sledge"
]

print("Length of Disco:", len(songSearchDisco))

listerDisco = []
for songSearch in songSearchDisco:
    x = getTracks(songSearch)
    listerDisco.append(x)

duplicateSongs(listerDisco)

Length of Disco: 50
No duplicates found


In [16]:
#50 songs - good
songSearchHipHop = [
    "N.Y. State of Mind Nas",
    "Electric Relaxation A Tribe Called Quest",
    "Gin and Juice Snoop Dogg",
    "Big Poppa The Notorious B.I.G.",
    "Protect Ya Neck Wu-Tang Clan",
    "The Message Nas",
    "Can't Truss It Public Enemy",
    "It Was A Good Day Ice Cube",
    "Keep Ya Head Up 2Pac",
    "Paid in Full Eric B. & Rakim",
    "The Breaks Kurtis Blow",
    "Hip Hop Hooray Naughty By Nature",
    "I Used To Love H.E.R. Common",
    "The Next Episode Dr. Dre",
    "Ms. Jackson OutKast",
    "My Name Is Eminem",
    "Fight The Power Public Enemy",
    "Jump Kris Kross",
    "I Got 5 On It Luniz",
    "You Can't Stop The Reign Shaquille O'Neal",
    "O.P.P. Naughty By Nature",
    "Hail Mary 2Pac",
    "C.R.E.A.M. Wu-Tang Clan",
    "Children's Story Slick Rick",
    "The Show Doug E. Fresh & Slick Rick",
    "Hard Knock Life (Ghetto Anthem) Jay-Z",
    "Rapper's Delight The Sugarhill Gang",
    "B.O.B Outkast",
    "The Way I Am Eminem",
    "The Bridge is Over Boogie Down Productions",
    "Da Rockwilder Method Man, Redman",
    "Can't Tell Me Nothing Kanye West",
    "Mass Appeal Gang Starr",
    "One Mic Nas",
    "Ruffneck MC Lyte",
    "Grindin' Clipse",
    "Put It On Big L",
    "Tha Crossroads Bone Thugs-N-Harmony",
    "X Gon' Give It To Ya DMX",
    "Nas Is Like Nas",
    "Passin' Me By The Pharcyde",
    "Ain't No Half-Steppin' Big Daddy Kane",
    "The World Is Yours Nas",
    "Ms. Fat Booty Mos Def",
    "Tha Shiznit Snoop Dogg",
    "Ice Cream Wu-Tang Clan",
    "Insane in the Brain Cypress Hill",
    "Criminal Eminem",
    "Super Bass Nicki Minaj",
    "Thrift Shop Macklemore"
]

print("Length of HipHop:", len(songSearchHipHop))

listerHipHop= []
for songSearch in songSearchHipHop:
    x = getTracks(songSearch)
    listerHipHop.append(x)

duplicateSongs(listerHipHop)

Length of HipHop: 50
No duplicates found


In [17]:
#50 - good
songSearchMetal = [
    "Master Of Puppets Metallica",
    "One Metallica",
    "Fade To Black Metallica",
    "Nothing Else Matters Metallica",
    "Paranoid Black Sabbath",
    "Iron Man Black Sabbath",
    "War Pigs Black Sabbath",
    "Mr. Crowley Ozzy Osbourne",
    "Bark At The Moon Ozzy Osbourne",
    "Ace Of Spades Motörhead",
    "Overkill Motörhead",
    "The Trooper Iron Maiden",
    "Run To The Hills Iron Maiden",
    "Fear Of The Dark Iron Maiden",
    "Number Of The Beast Iron Maiden",
    "Breaking The Law Judas Priest",
    "Painkiller Judas Priest",
    "Electric Eye Judas Priest",
    "Living After Midnight Judas Priest",
    "Angel Of Death Slayer",
    "Raining Blood Slayer",
    "Seasons In The Abyss Slayer",
    "South Of Heaven Slayer",
    "Chop Suey! System Of A Down",
    "Toxicity System Of A Down",
    "Aerials System Of A Down",
    "BYOB System Of A Down",
    "Black Hole Sun Soundgarden",
    "Fell On Black Days Soundgarden",
    "Rusty Cage Soundgarden",
    "Outshined Soundgarden",
    "Symphony Of Destruction Megadeth",
    "Peace Sells Megadeth",
    "Hangar 18 Megadeth",
    "Killing In The Name Rage Against The Machine",
    "Bombtrack Rage Against The Machine",
    "Testify Rage Against The Machine",
    "Know Your Enemy Rage Against The Machine",
    "Walk Pantera",
    "Mouth For War Pantera",
    "Refuse/Resist Sepultura",
    "Roots Bloody Roots Sepultura",
    "Attitude Sepultura",
    "Funeral Fog Mayhem",
    "Left Hand Path Entombed", 
    "Slaughter of the Soul At The Gates",
    "Got the Time Anthrax",
    "Are You Dead Yet Children of Bodom",
    "Devil's Plaything Danzig",
    "Dismembered Dismember"
]

print("Length of Metal:", len(songSearchMetal))

listerMetal= []
for songSearch in songSearchMetal:
    x = getTracks(songSearch)
    listerMetal.append(x)

duplicateSongs(listerMetal)

Length of Metal: 50
No duplicates found


In [18]:
#50 songs - good
songSearchJazz = [
    "Take Five Dave Brubeck",
    "What A Wonderful World Louis Armstrong",
    "Summertime Ella Fitzgerald",
    "My Favorite Things John Coltrane",
    "Take the A Train Duke Ellington",
    "In A Sentimental Mood Duke Ellington",
    "Autumn Leaves Cannonball Adderley",
    "Round Midnight Thelonious Monk",
    "Don't Worry Be Happy Bobby McFerrin",
    "Strange Fruit Billie Holiday",
    "Feeling Good Nina Simone",
    "Birdland Weather Report",
    "Minnie The Moocher Cab Calloway",
    "All Blues Miles Davis",
    "Take The 'A' Train Ella Fitzgerald",
    "Ain't Misbehavin' Fats Waller",
    "Lullaby Of Birdland Ella Fitzgerald",
    "The Nearness Of You Ella Fitzgerald",
    "Mood Indigo Duke Ellington",
    "Georgia On My Mind Ray Charles",
    "Desafinado João Gilberto",
    "Straighten Up And Fly Right Nat King Cole",
    "Django Modern Jazz Quartet",
    "Mack The Knife Louis Armstrong",
    "Watermelon Man Herbie Hancock",
    "The Look Of Love Sergio Mendes & Brasil '66",
    "So What Miles Davis",
    "Spain Chick Corea",
    "A Night In Tunisia Dizzy Gillespie",
    "Body And Soul Coleman Hawkins",
    "St. Louis Blues Bessie Smith",
    "Goodbye Pork Pie Hat Charles Mingus",
    "Blue Train John Coltrane",
    "Someday My Prince Will Come Miles Davis",
    "West Coast Blues Wes Montgomery",
    "Round About Midnight Miles Davis",
    "Cheek To Cheek Ella Fitzgerald",
    "My One And Only Love John Coltrane",
    "Boplicity Miles Davis",
    "Song For My Father Horace Silver",
    "Take The Coltrane Duke Ellington, John Coltrane",
    "Birds of Fire Mahavishnu Orchestra",
    "It's Only A Paper Moon Ella Fitzgerald",
    "Sister Sadie Horace Silver",
    "Afro Blue John Coltrane",
    "Blue In Green Miles Davis",
    "Naima John Coltrane",
    "Cantaloupe Island Herbie Hancock",
    "Chameleon Herbie Hancock",
    "Lonely Woman Ornette Coleman"
]

print("Length of Jazz:", len(songSearchJazz))

listerJazz= []
for songSearch in songSearchJazz:
    x = getTracks(songSearch)
    listerJazz.append(x)

duplicateSongs(listerJazz)

Length of Jazz: 50
No duplicates found


In [19]:
#50 songs - good
songSearchPop = [
    "Levitating Dua Lipa",
    "Peaches Justin Bieber",
    "Shape of You Ed Sheeran",
    "Uptown Funk Mark Ronson",
    "Closer The Chainsmokers"
    "Say You Won't Let Go James Arthur",
    "Thinking Out Loud Ed Sheeran",
    "Hello Adele",
    "Blinding Lights The Weeknd",
    "Don't Start Now Dua Lipa",
    "Happy Pharrell Williams"
    "Watermelon Sugar Harry Styles",
    "Beautiful Mistakes Maroon 5",
    "Heat Waves Glass Animals",
    "Radioactive Imagine Dragons",
    "Shallow Lady Gaga", 
    "Someone Like You Adele",
    "All of Me John Legend",
    "Love Yourself Justin Bieber",
    "Counting Stars OneRepublic",  
    "Stay Justin Bieber",
    "No More Tears Left to Cry Ariana Grande",
    "We Found Love Rihanna", 
    "Stay with Me Sam Smith",
    "Roar Katy Perry",
    "Sugar Maroon 5",
    "Bad Guy Billie Eilish",
    "Shake It Off Taylor Swift",
    "Cheap Thrills Sia",
    "Girls Like You Maroon 5",
    "Rolling in the Deep Adele",
    "That's What I Like Bruno Mars",
    "Classic MKTO",
    "Love You Like a Love Song Selena Gomez",
    "Just Dance Lady Gaga",
    "Disturbia Rihanna",
    "Call Me Maybe Carly Rae Jepsen",
    "Bleeding Love Leona Lewis",
    "Bang Bang Jessie J",
    "Attention Charlie Puth",
    "Paradise Coldplay",
    "Tonight Tonight Hot Chelle Rae",
    "Dynamite Taio Cruz",
    "Cooler Than Me Mike Posner",
    "Royals Lorde",
    "See You Again Charlie Puth",
    "Forget You Cee Lo Green",
    "Let Her Go Passenger",
    "I Won't Give Up Jason Mraz",
    "Fireflies Owl City",
    "Wrecking Ball Miley Cyrus",
    "Somebody That I Used To Know Gotye"
]
print("Length of Pop:", len(songSearchPop))

listerPop= []
for songSearch in songSearchPop:
    x = getTracks(songSearch)
    listerPop.append(x)

duplicateSongs(listerPop)

Length of Pop: 50
No duplicates found


In [20]:
#50 songs - good
songSearchReggae = [
    "Do It and Done Pressure",
    "Kingston Town Alborosie",
    "Here Comes Trouble Chronixx",
    "Thank You Mama Sizzla",
    "Affairs of the Heart Damian Marley",
    "What One Dance Can Do Beres Hammond",
    "Blackheart Man Bunny Wailer",
    "Redemption Song Bob Marley & The Wailers",
    "Is This Love Bob Marley & The Wailers",
    "Three Little Birds Bob Marley & The Wailers",
    "Stir It Up Bob Marley & The Wailers",
    "Jamming Bob Marley & The Wailers",
    "Buffalo Soldier Bob Marley & The Wailers",
    "Could You Be Loved Bob Marley & The Wailers",
    "Waiting In Vain Bob Marley & The Wailers",
    "Get Up, Stand Up The Wailers",
    "I Shot The Sheriff Bob Marley & The Wailers",
    "Roots, Rock, Reggae Bob Marley & The Wailers",
    "Sweat (A La La La La Long) Inner Circle",
    "Night Nurse Gregory Isaacs",
    "Here Comes the Hotstepper Ini Kamoze",
    "Legalize It Peter Tosh",
    "54-46 That's My Number Toots & The Maytals",
    "Rivers of Babylon Boney M.",
    "Satta Massagana The Abyssinians",
    "Bam Bam Sister Nancy",
    "Oh Carolina Shaggy",
    "Pressure Drop Toots & The Maytals",
    "Israelites Desmond Dekker & The Aces",
    "Uptown Top Ranking Althea & Donna",
    "Simmer Down The Wailers",
    "Cherry Oh Baby Eric Donaldson",
    "Police and Thieves Junior Murvin",
    "The Harder They Come Jimmy Cliff",
    "Marcus Garvey Burning Spear",
    "Johnny Too Bad The Slickers",
    "Small Axe Bob Marley & The Wailers",
    "Natty Dread Bob Marley & The Wailers",
    "Marcus Garvey Culture",
    "Pass The Dutchie Musical Youth",
    "Mad Over You Runtown",
    "Kingston Town UB40",
    "Wonderful World, Beautiful People Jimmy Cliff",
    "Funky Kingston Toots & The Maytals",
    "One Drop Bob Marley & The Wailers",
    "Nah Let Go Gyptian",
    "Mek Dem Know Yellowman",
    "Telephone Love J.C. Lodge",
    "Better Must Come Delroy Wilson",
    "Oh Me Oh My Ernie Smith"
]

print("Length of Reggae:", len(songSearchReggae))

listerReggae = []
for songSearch in songSearchReggae:
    x = getTracks(songSearch)
    listerReggae.append(x)

duplicateSongs(listerReggae)

Length of Reggae: 50
No duplicates found


In [21]:
#50 songs - good
songSearchRock = [
    "Sweet Child O' Mine Guns N' Roses",
    "Back In Black AC/DC",
    "Stairway To Heaven Led Zeppelin",
    "Hotel California Eagles",
    "Don't Stop Believin' Journey",
    "Livin' On A Prayer Bon Jovi",
    "Dream On Aerosmith",
    "Crazy Train Ozzy Osbourne",
    "Sweet Home Alabama Lynyrd Skynyrd",
    "More Than A Feeling Boston",
    "Jump Van Halen",
    "Paradise City Guns N' Roses",
    "Black Dog Led Zeppelin",
    "The Chain Fleetwood Mac",
    "We Will Rock You Queen",
    "Born To Run Bruce Springsteen",
    "Pour Some Sugar On Me Def Leppard",
    "Eye Of The Tiger Survivor",
    "You Give Love A Bad Name Bon Jovi",
    "Don't Fear The Reaper Blue Öyster Cult",
    "Whole Lotta Love Led Zeppelin",
    "Thunderstruck AC/DC",
    "November Rain Guns N' Roses",
    "Every Breath You Take The Police",
    "All Along The Watchtower Jimi Hendrix",
    "Living After Midnight Judas Priest",
    "Runnin' With The Devil Van Halen",
    "Proud Mary Creedence Clearwater Revival",
    "Comfortably Numb Pink Floyd",
    "Everybody Wants Some Van Halen",
    "Smoke on the Water Deep Purple",
    "Walk This Way Aerosmith",
    "Enter Sandman Metallica",
    "Can't Stop Red Hot Chili Peppers",
    "Kashmir Led Zeppelin", 
    "Free Bird Lynyrd Skynyrd",
    "Purple Haze Jimi Hendrix",
    "American Girl Tom Petty and the Heartbreakers"
    "Dancing in the Dark Bruce Springsteen",
    "Satisfaction Rolling Stones",
    "Bohemian Rhapsody Queen",
    "Like a Rolling Stone Bob Dylan",
    "Smells Like Teen Spirit Nirvana",
    "Johnny B. Goode Chuck Berry",
    "Strawberry Fields Forever",
    "Rock You Like a Hurricane Scorpions",
    "You Shook Me All Night Long AC/DC",
    "Here Comes the Sun The Beatles",
    "Free Fallin' Tom Petty",
    "Wonderwall Oasis",
    "Africa Toto"
]
print("Length of Rock:", len(songSearchRock))

listerRock = []
for songSearch in songSearchRock:
    x = getTracks(songSearch)
    listerRock.append(x)

duplicateSongs(listerRock)

Length of Rock: 50
No duplicates found


In [22]:

featureValueList = [["song_name", "artist_name", "song_id", "analysis_sample_rate", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "bpm", "duration", "time_signature", "genre"]]
combined_dict = {}

def addSongtoList(comboDict, genre_name, featList, listers):
    for ID in listers:
        track = sp.track(ID)
        song_name = track['name']
        artist_name = track['artists'][0]['name']
        d0 = {"song_name": song_name, "artist_name": artist_name}
        d1 = getAudioAnalysis(ID)
        d2 = getAudioFeatures(ID)
        d3 = {"genre": genre_name}
        d2.update(d3)
        d1.update(d2)
        d0.update(d1)
        comboDict.update(d0)
        values_list = list(comboDict.values())
        featList.append(values_list)
    return featList


#end of it all
listofGenres_IDs = [listerBlues, listerClassical, listerCountry, listerDisco, listerHipHop, listerJazz, listerMetal, listerPop, listerReggae, listerRock]
listofGenres = ["Blues", "Classical", "Country", "Disco", "HipHop", "Jazz", "Metal", "Pop", "Reggae", "Rock"]

In [23]:
for genreList, genreName in zip(listofGenres_IDs, listofGenres):
    totList = addSongtoList(combined_dict, genreName, featureValueList, genreList)

In [24]:
print("Number of Songs:", len(totList[1:]))

Number of Songs: 500


In [25]:
file = "../Data/features_spotify.csv"
with open(file, 'w', newline='') as file:
    writer = csv.writer(file)
    for row in totList:
        writer.writerow(row)